In [1]:
# Add the dependencies

import pandas as pd
import numpy as np
from citipy import citipy
import requests
import time
from config import weather_api_key
from datetime import datetime

In [2]:
# Restrict latitudes to areas of human habitation... omit high arctic and antarctic

lats = np.random.uniform(low=-60.000, high=85.000, size=3000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=3000)

lat_lngs = zip(lats,lngs)

In [3]:
# Add the latitudes and longitudes to a list.

coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the lat and lngs combinations.

for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-37.42878336755108 -5.90830517305551
9.800703374183058 125.12639965579092
20.492308973574424 -59.874170579929824
-32.138388238815764 43.38382110014399
17.479722281260237 57.271724098644654
-7.959333258625726 23.537643581449174
6.8366029301438545 170.01340798855819
41.1110344059136 105.12019651076002
-30.888106407239295 41.51989656879775
-42.33912314978972 -110.49096868555267
-21.46721724544225 33.582746543407524
3.9590879281360216 29.11982917551248
69.89546084882573 -42.86223499361239
34.585412996376775 124.31477134007037
-50.42980942055212 98.31121795401515
-40.99178313624414 -51.5460807943752
37.68357725793609 -36.71280895563777
77.25844503072344 30.03935568163942
-44.61464687990037 -173.20739097816627
39.326387485876836 5.106670170238061
64.21149618676185 119.55648118944896
-16.154569707189587 16.66200686831661
72.20284882910255 23.2138505673702
16.42670689883522 146.10758661432283
-55.55717357912442 92.80082660314099
17.21633947068301 -59.380072826348155
-2.372134377649644 -37.6881

24.088018933455913 6.843024267617409
-3.786381716756864 -162.072085731886
51.64346068173964 170.91614414658926
-18.64894940276305 -32.73471226832305
22.512226929733075 24.132832085502486
78.61426154817642 -56.114935276457075
-15.7540650289423 -94.37868761578582
5.675814362489646 -151.39910610397553
40.35710086759944 179.3357121825985
46.31379524505074 -39.1586831093654
27.39272383246751 -31.350425634211035
-24.298318395270293 132.81744733569974
3.4334122810752135 90.95508809296024
35.72116053285123 -153.13696134325218
63.118480505265296 -136.84569907856948
-32.65639799163636 17.486599557672747
10.457230281338852 11.576389580196633
1.006530509827357 -114.93611931094853
76.45359664814933 87.31809712000216
24.71260288599673 158.19630287589405
18.024650739197682 101.65546659055036
-48.429887731907996 31.09987623174871
61.90626606832636 176.596212886552
-40.87482627594122 35.021210267067175
-26.024467790559513 151.3052703184169
77.2813832846627 -46.02843357537458
-32.492659084707725 -68.476

In [5]:
# Create a list for holding the cities

cities = []

# Identify the nearest city for each lat-lngs combination

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # If the city is unique - add it to the list of cities
    
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm processing
print(len(cities))

1190


In [6]:
# Create an empty list to hold the weather data
city_data = []

# Provide the base URL for collecting weather data
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial" + "&appid=" + weather_api_key

# Build our log
print('Beginning Data Retrieval        ')
print('--------------------------------')

# Create counters

record_count = 1
set_count = 1

# Loop through all the cities in the list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 to prevent weather data collection timeouts
    if (i % 50 == 0 and i >= 50):
        set_count +=1
        record_count = 1
        time.sleep(60)
        
    # Create the endpoint URL for each city.
    city_url = url + '&q=' + city.replace(' ','+')
    
    # Log the URL, record, and set numbers and the city.
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    
    # Increment the record count
    record_count +=1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the desired data.
        
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_weather_description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO Standard.
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%M-%D %H:%M:%S')
        
        # Append the data into the city_data list.
        
        city_data.append({'City': city.title(),
                         'Country': city_country,
                         'Date': city_date,
                         'Lat': city_lat,
                         'Lng': city_lng,
                         'Max Temp': city_max_temp,
                         'Humidity': city_humidity,
                         'Cloudiness': city_clouds,
                         'Wind Speed': city_wind,
                         'Weather Description': city_weather_description})
        
# If an error is detected, skip the city.
    except:
        print(f'City not found.  Skipping ....')
        pass
    
# Indicate that Data loading is complete.

print('--------------------------------')
print('Data Retrieval Completed        ')
print('--------------------------------')

Beginning Data Retrieval        
--------------------------------
Processing Record 1 of Set 1 | jamestown
Processing Record 2 of Set 1 | san francisco
Processing Record 3 of Set 1 | codrington
Processing Record 4 of Set 1 | tsihombe
City not found.  Skipping ....
Processing Record 5 of Set 1 | salalah
Processing Record 6 of Set 1 | kaniama
Processing Record 7 of Set 1 | butaritari
Processing Record 8 of Set 1 | haibowan
City not found.  Skipping ....
Processing Record 9 of Set 1 | beloha
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | chipinge
Processing Record 12 of Set 1 | yambio
Processing Record 13 of Set 1 | tasiilaq
Processing Record 14 of Set 1 | yatou
Processing Record 15 of Set 1 | busselton
Processing Record 16 of Set 1 | mar del plata
Processing Record 17 of Set 1 | ribeira grande
Processing Record 18 of Set 1 | berlevag
Processing Record 19 of Set 1 | vaini
Processing Record 20 of Set 1 | mahon
Processing Record 21 of Set 1 | verkhnevilyuysk
Process

Processing Record 35 of Set 4 | nishihara
Processing Record 36 of Set 4 | alofi
Processing Record 37 of Set 4 | garowe
Processing Record 38 of Set 4 | mataura
Processing Record 39 of Set 4 | srandakan
Processing Record 40 of Set 4 | ahipara
Processing Record 41 of Set 4 | puranpur
Processing Record 42 of Set 4 | qaanaaq
Processing Record 43 of Set 4 | kirakira
Processing Record 44 of Set 4 | tilichiki
Processing Record 45 of Set 4 | micheweni
Processing Record 46 of Set 4 | shenjiamen
Processing Record 47 of Set 4 | cayenne
Processing Record 48 of Set 4 | saleaula
City not found.  Skipping ....
Processing Record 49 of Set 4 | jaumave
Processing Record 50 of Set 4 | cabadiangan
Processing Record 1 of Set 5 | altay
Processing Record 2 of Set 5 | klaksvik
Processing Record 3 of Set 5 | ortigueira
Processing Record 4 of Set 5 | chokurdakh
Processing Record 5 of Set 5 | tazovskiy
Processing Record 6 of Set 5 | vilyuysk
Processing Record 7 of Set 5 | lompoc
Processing Record 8 of Set 5 | los

Processing Record 24 of Set 8 | asau
Processing Record 25 of Set 8 | lakki marwat
Processing Record 26 of Set 8 | lolua
City not found.  Skipping ....
Processing Record 27 of Set 8 | madaoua
Processing Record 28 of Set 8 | palu
Processing Record 29 of Set 8 | huangzhai
Processing Record 30 of Set 8 | lagos
Processing Record 31 of Set 8 | bathsheba
Processing Record 32 of Set 8 | talnakh
Processing Record 33 of Set 8 | praia
Processing Record 34 of Set 8 | yurga
Processing Record 35 of Set 8 | iquitos
Processing Record 36 of Set 8 | adrar
Processing Record 37 of Set 8 | bambous virieux
Processing Record 38 of Set 8 | san lawrenz
Processing Record 39 of Set 8 | pundaguitan
Processing Record 40 of Set 8 | pangkalanbuun
Processing Record 41 of Set 8 | morondava
Processing Record 42 of Set 8 | punta arenas
Processing Record 43 of Set 8 | rawson
Processing Record 44 of Set 8 | portland
Processing Record 45 of Set 8 | otjimbingwe
Processing Record 46 of Set 8 | coahuayana
Processing Record 47

Processing Record 10 of Set 12 | tarauaca
Processing Record 11 of Set 12 | tarabuco
Processing Record 12 of Set 12 | canete
Processing Record 13 of Set 12 | hervey bay
Processing Record 14 of Set 12 | vao
Processing Record 15 of Set 12 | sharlyk
Processing Record 16 of Set 12 | nome
Processing Record 17 of Set 12 | fevralsk
City not found.  Skipping ....
Processing Record 18 of Set 12 | camacupa
Processing Record 19 of Set 12 | bajo baudo
City not found.  Skipping ....
Processing Record 20 of Set 12 | nigde
Processing Record 21 of Set 12 | rubtsovsk
Processing Record 22 of Set 12 | eskasem
City not found.  Skipping ....
Processing Record 23 of Set 12 | ca mau
Processing Record 24 of Set 12 | kishorganj
Processing Record 25 of Set 12 | teluknaga
Processing Record 26 of Set 12 | coihaique
Processing Record 27 of Set 12 | mzimba
Processing Record 28 of Set 12 | villarrica
Processing Record 29 of Set 12 | salinas
Processing Record 30 of Set 12 | salmas
Processing Record 31 of Set 12 | buch

Processing Record 41 of Set 15 | tatsinskaya
City not found.  Skipping ....
Processing Record 42 of Set 15 | grindavik
Processing Record 43 of Set 15 | sabha
Processing Record 44 of Set 15 | granadilla de abona
Processing Record 45 of Set 15 | shache
Processing Record 46 of Set 15 | nacozari
Processing Record 47 of Set 15 | aztec
Processing Record 48 of Set 15 | vardo
Processing Record 49 of Set 15 | bhabua
Processing Record 50 of Set 15 | valleyview
Processing Record 1 of Set 16 | svetlyy
Processing Record 2 of Set 16 | houma
Processing Record 3 of Set 16 | walvis bay
Processing Record 4 of Set 16 | puquio
Processing Record 5 of Set 16 | gangtok
Processing Record 6 of Set 16 | chumikan
Processing Record 7 of Set 16 | oneonta
Processing Record 8 of Set 16 | college
Processing Record 9 of Set 16 | beyneu
Processing Record 10 of Set 16 | iwanai
Processing Record 11 of Set 16 | port hedland
Processing Record 12 of Set 16 | luangwa
Processing Record 13 of Set 16 | bartica
Processing Record

Processing Record 28 of Set 19 | adelaide
Processing Record 29 of Set 19 | biak
Processing Record 30 of Set 19 | vallenar
Processing Record 31 of Set 19 | fare
Processing Record 32 of Set 19 | ayagoz
Processing Record 33 of Set 19 | manassas park
Processing Record 34 of Set 19 | bom jesus
Processing Record 35 of Set 19 | koszalin
Processing Record 36 of Set 19 | doha
Processing Record 37 of Set 19 | sparks
Processing Record 38 of Set 19 | kashi
Processing Record 39 of Set 19 | warrnambool
Processing Record 40 of Set 19 | manggar
Processing Record 41 of Set 19 | oksfjord
Processing Record 42 of Set 19 | kilindoni
Processing Record 43 of Set 19 | ponta delgada
Processing Record 44 of Set 19 | morro bay
Processing Record 45 of Set 19 | rogers
Processing Record 46 of Set 19 | biltine
Processing Record 47 of Set 19 | lloret de mar
Processing Record 48 of Set 19 | mogadishu
Processing Record 49 of Set 19 | songjianghe
Processing Record 50 of Set 19 | shizunai
Processing Record 1 of Set 20 | 

Processing Record 16 of Set 23 | tsumeb
Processing Record 17 of Set 23 | cangzhou
Processing Record 18 of Set 23 | pilao arcado
City not found.  Skipping ....
Processing Record 19 of Set 23 | winneba
Processing Record 20 of Set 23 | valley city
Processing Record 21 of Set 23 | alotau
City not found.  Skipping ....
Processing Record 22 of Set 23 | phonhong
Processing Record 23 of Set 23 | cabatuan
Processing Record 24 of Set 23 | svetlaya
Processing Record 25 of Set 23 | bougouni
Processing Record 26 of Set 23 | ternova
Processing Record 27 of Set 23 | puteyets
Processing Record 28 of Set 23 | ishinomaki
Processing Record 29 of Set 23 | rocha
Processing Record 30 of Set 23 | san antonio
Processing Record 31 of Set 23 | atyrau
Processing Record 32 of Set 23 | barillas
Processing Record 33 of Set 23 | lingdong
Processing Record 34 of Set 23 | ambunti
Processing Record 35 of Set 23 | solano
Processing Record 36 of Set 23 | charters towers
Processing Record 37 of Set 23 | leopold
Processing

In [7]:
city_data_df = pd.DataFrame(city_data)

city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,Jamestown,US,2022-25-09/02/22 21:25:31,42.0970,-79.2353,82.02,39,0,10.36,clear sky
1,San Francisco,US,2022-27-09/02/22 21:27:18,37.7749,-122.4194,96.03,74,20,28.77,few clouds
2,Codrington,AU,2022-27-09/02/22 21:27:23,-38.2667,141.9667,48.06,87,77,8.75,broken clouds
3,Salalah,OM,2022-27-09/02/22 21:27:24,17.0151,54.0924,78.89,83,75,5.75,broken clouds
4,Kaniama,CD,2022-27-09/02/22 21:27:24,-7.5667,24.1833,68.59,87,100,6.76,light rain
...,...,...,...,...,...,...,...,...,...,...
1094,Ghanzi,BW,2022-54-09/02/22 21:54:30,-21.5667,21.7833,58.32,21,0,9.01,clear sky
1095,Banes,CU,2022-56-09/02/22 21:56:29,20.9625,-75.7186,82.81,70,85,13.60,overcast clouds
1096,Giyani,ZA,2022-56-09/02/22 21:56:29,-23.3025,30.7187,61.32,39,0,3.60,clear sky
1097,Katobu,ID,2022-56-09/02/22 21:56:30,-4.9408,122.5276,73.85,94,16,4.09,few clouds


In [8]:
# Create the output (export) file (CSV)

output_data_file = 'WeatherPy_Database.csv'

# Export the city data DataFrame to a CSV

city_data_df.to_csv(output_data_file)
